In [19]:
import json
import nltk
import gzip
from modules import bpencoding

In [20]:
with open('../data/simplewiki/simplewiki-20171103.encoder_table_10k.txt', 'rt', encoding='utf-8') as f:
    lines = f.readlines()
table = [l.strip() for l in lines]

In [21]:
encoder = bpencoding.Encoder(table)

In [36]:
encoder.encode('cancers')

[WordFragment(index=2811, text='cancer'), WordFragment(index=10, text='s')]